# 26 Point 2

## Step 0: Utils for Jupyter Notebooks

### 1.1.1 Add Show/Hide code button

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [2]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate Models + Props

In [38]:
# Edit this cell to define parameters for your model...

strat_1 = 4 # the strategy that player 1 uses
strat_2 = 1 # strategy for player 2

print("CELL 8")

CELL 8


In [39]:
%%writetemplate 26Point2/params/26Point2_1_1.csv
players, 2
no_sides, 4
no_dice, 4
p1, 1
p2, 2
strategy1, {strat_1}
strategy2, {strat_2}

In [44]:
def export_csv(strat_1, strat_2):
    with open(f"26Point2/params/26Point2_{strat_1}_{strat_2}.csv", 'w') as f:
        f.write("players, 2")
        f.write("no_sides, 4")
        f.write("no_dice, 4")
        f.write("p1, 1")
        f.write("p2, 2")
        f.write(f"strategy1, {strat_1}")
        f.write(f"strategy2, {strat_2}")

In [45]:
def build_model(strat_1, strat_2):
    export_csv(strat_1, strat_2)

    model_spec = f"26Point2_{strat_1}_{strat_2}"

    !mkdir "26Point2/models"
    output_model = f"26Point2/models/{model_spec}.prism"

    !python utils/pyprism.py 26Point2/26Point2.pyprism $output_model 26Point2/params/26Point2_auto_params.csv

In [46]:
from IPython.display import Javascript


strats = [[1, 1], [2,1], [3,1], [4,1], [5,1]]

for strat in strats:
    strat_1, strat_2 = strat
    build_model(strat_1, strat_2)

A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_1_1.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_2_1.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_3_1.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_4_1.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_5_1.prism


# Properties to consider

* Win rate with various strategies (pick 2, pick 3, pick 4, pick 3/4)

* Adversary to pick 2/pick 4, pick3/4) (figure out how long strategy generation takes...)

* Potentially figure out bust rate? Though it may be nicer to do this theoretically for non-hybrid strategies...

In [ ]:
strategies = []
# strats have (model_spec, no_spaces, prop_number, log_filename)

strategies.append(["1_1", 50, 1, "random_winrate_50.log"])
strategies.append(["4_1", 50, ])

## Step 2a: Make auto.sh file

The big challenge with this case study is that the models are far too large to examine normally. Instead, we create a shell file that can run on a computing cluster.

In [22]:
%%writetemplate 26Point2/logs/auto_cluster.sh
#/bin/csh

~/prism-games-csg-dev/prism/bin/prism {model_spec}.prism 26Point2_manual.props -javamaxmem 700g -const no_spaces=50 -prop 1 > spaces50_prop1.log

## Step 2b: Make logs with statistical model checking.

Statistical model checking isn't great, but it's far quicker than probabilistic model checking so it's appropriate for "quick checking" purposes.

In [23]:


!prism 26Point2/models/{model_spec}.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 5 -sim -simmethod ci -simsamples 10000

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Mon Mar 01 14:52:00 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games 26Point2/models/26Point2_4_1.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 5 -sim -simmethod ci -simsamples 10000

Parsing model file "26Point2/models/26Point2_4_1.prism"...

Type:        CSG
Modules:     player1 player2 board 
Variables:   p1_pos p1_chosen_dice p1_1s p1_2s p1_3s p1_4s p2_pos p2_chosen_dice p2_1s p2_2s p2_3s p2_4s state winner 

Switching to explicit engine, which supports CSGs...

Parsing properties file "26Point2/props/26Point2_manual.props"...

7 properties:
(1) Pmax=? [ F "p1_wins" ]
(2) Pmax=? [ F "p2_wins" ]
(3) Pmax=? [ F "players_tie" ]
(4) Pmax=? [ F "game_over" ]
(5) R{"no_rounds"}max=? [ F "game_over" ]
(6) R{"p1_busts"}max=? [ F "game_over" ]
(7) R{"p1_busts"}max=? [ F "game_over" ]/R{"no_rounds"}max=? [ F "game_over" ]

------------------------------------------

In [24]:
!prism 26Point2/models/{model_spec}.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 6 -sim -simmethod ci -simsamples 10000

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Mon Mar 01 14:52:26 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games 26Point2/models/26Point2_4_1.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 6 -sim -simmethod ci -simsamples 10000

Parsing model file "26Point2/models/26Point2_4_1.prism"...

Type:        CSG
Modules:     player1 player2 board 
Variables:   p1_pos p1_chosen_dice p1_1s p1_2s p1_3s p1_4s p2_pos p2_chosen_dice p2_1s p2_2s p2_3s p2_4s state winner 

Switching to explicit engine, which supports CSGs...

Parsing properties file "26Point2/props/26Point2_manual.props"...

7 properties:
(1) Pmax=? [ F "p1_wins" ]
(2) Pmax=? [ F "p2_wins" ]
(3) Pmax=? [ F "players_tie" ]
(4) Pmax=? [ F "game_over" ]
(5) R{"no_rounds"}max=? [ F "game_over" ]
(6) R{"p1_busts"}max=? [ F "game_over" ]
(7) R{"p1_busts"}max=? [ F "game_over" ]/R{"no_rounds"}max=? [ F "game_over" ]

------------------------------------------

## Step 3: Store data

Once we have the log files from the cluster, we then filter those files and generate more readable data. (Note that we perform this filtering at this stage, not on the cluster - that way, we might not need to run everything again if we need more data than we anticipated.